# Investigate the deeplabcut infrastructure modules
This is a notebook where I am going through the code in DLC to understand it. We will use the Reaching Mackenzie demo data (provided with DeepLabCut). 

* Load modules
* Load data

In [1]:
import deeplabcut
print(deeplabcut.__version__)
from pathlib import Path
import os.path
import tensorflow as tf
print(tf.__version__)

2.0.4.1
1.10.0


In [5]:
# https://github.com/ipython/ipython/issues/10123#issuecomment-354889020
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
print('nb_dir: ' + nb_dir)
os.chdir(nb_dir)

exp_id = "Demo"
experiment = {"Demo": ["Reaching-Mackenzie-2018-08-30", "ReachingAug30-trainset95shuffle1"],
              "FSL": ["Fly-Sayed-2019-02-20", "FlyFeb20-trainset95shuffle1"]}

experiment = experiment[exp_id]
main_path=os.path.join(nb_dir, experiment[0])
config_path = os.path.join(main_path, "config.yaml")
poseconfigfile=Path(os.path.join(main_path, "dlc-models/iteration-0/", experiment[1], "train/pose_cfg.yaml"))
config_yaml = poseconfigfile
print(config_path)
print(config_yaml)


nb_dir: /home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples
/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/config.yaml
/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/pose_cfg.yaml


# The `train()` function in `pose_estimation_tensorflow/train.py`

deeplabcut.train_network() is just a CLI interface for the most common call to train(). Let us go deep into its code in order to understand it!!!

### Prepare variables inside the function scope

In [13]:
shuffle=1
trainingsetindex=0
# gputouse=None
max_snapshots_to_keep=5
max_to_keep=max_snapshots_to_keep
autotune=False

### Function arguments
displayiters=10
display_iters=displayiters
# saveiters=1000
saveiters=100
save_iters=saveiters
# maxiters=200000
maxiters=100

max_iter=maxiters

import logging, os
os.chdir(str(Path(config_yaml).parents[0])) #switch to folder of config_yaml (for logging)
from deeplabcut.pose_estimation_tensorflow.util.logging import setup_logging
# def setup_logging():
#     FORMAT = '%(asctime)-15s %(message)s'
#     logging.basicConfig(filename=os.path.join('log.txt'), filemode='w',
#                         datefmt='%Y-%m-%d %H:%M:%S',
#                         level=logging.INFO, format=FORMAT)

#     console = logging.StreamHandler()
#     console.setLevel(logging.INFO)
#     logging.getLogger('').addHandler(console)

setup_logging()

FileNotFoundError: [Errno 2] No such file or directory: '/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train'

In [12]:
start_path=os.getcwd()
print(start_path)
Path(config_yaml)

/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples


PosixPath('/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/pose_cfg.yaml')

## How is cfg created in train() @ pose_estimation_tensorflow/train.py?

In [6]:
cfg = deeplabcut.pose_estimation_tensorflow.config.load_config(config_yaml)

Config:
{'all_joints': [[0]],
 'all_joints_names': ['Separator'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_FlyFeb20/Fly_Sayed95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_FlyFeb20/Documentation_data-Fly_95shuffle1.pickle',
 'minsize': 100,
 'mirror': False,
 'multi_step': [[0.005, 10000],
                

1. cfg is the output of `pose_estimation_tensorflow.config.load_config(path/config.yaml)`
2. `load_config()` is a call to `pose_estimation_tensorflow.config.cfg_from_file()` with the only argument it has set to: `"pose_cfg.yaml"`
3. `cfg_from_file`:
  * reads the pose_config.yaml file into an EasyDict
  * sets the snapshot_prefix to a default (depends on the trainpath, not default)
  * merges the pose_config with the default configurations (extracted via `cfg = auxiliaryfunctions.read_config(config)`)
  * merging is done via `pose_estimation_tensorflow.config._merge_a_into_b(a, b)`, a recursive function that calls itself!

4. Again in train(), the batch_size is set to a default of 1

## What is the dataset object?

In [7]:
from deeplabcut.pose_estimation_tensorflow.dataset.factory import create as create_dataset
dataset = create_dataset(cfg)
print("The current dataset has {} images".format(dataset.num_images))

i=0
dataset_array=dataset.raw_data["dataset"]
sample=dataset_array[0,i]
# len(sample)
# print(sample[0][0]) # path
# sample[1][0] # size [channel, h, w]
# joints = sample[2][0][0] # features [[id, x, y],
#                          #           [id, x, y]]

The current dataset has 18 images


In [8]:
dataset.raw_data["dataset"][:,0][0][2]

array([[array([[  0, 191,  41]])]], dtype=object)

Instantiates a `PoseDataset` object with the configuration supplied in cfg
It will have
1. Metadata attributes describing the dataset
2. A raw_data attribute with metadata and another attribute called dataset. This contains metadata about the images!
  * It's an array of size (1, num_images)
  * Each ith element is a list like object (np.void) for the ith image
  * [0][0] gives the path to the image
  * [0][1] is the size (channels, h, w)
  * [0][2] is the features (joints) in an array of shape nx3 where n is the number of features (feature_id, x, y)


## What is batch_spec?

In [9]:
from deeplabcut.pose_estimation_tensorflow.nnet.pose_net import get_batch_spec

# from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch
# class Batch(Enum):
#     inputs = 0
#     part_score_targets = 1
#     part_score_weights = 2
#     locref_targets = 3
#     locref_mask = 4
#     data_item = 5

    
# def get_batch_spec(cfg):
#     num_joints = cfg.num_joints
#     batch_size = cfg.batch_size
#     return {
#         Batch.inputs: [batch_size, None, None, 3],
#         Batch.part_score_targets: [batch_size, None, None, num_joints],
#         Batch.part_score_weights: [batch_size, None, None, num_joints],
#         Batch.locref_targets: [batch_size, None, None, num_joints * 2],
#         Batch.locref_mask: [batch_size, None, None, num_joints * 2]
#     }

batch_spec = get_batch_spec(cfg)

*batch* *specifications*: Consists of a dictionary with 5 entries, each storing a list of length 4.
The keys in this dictionary are Enum objects defined in:

`from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch`

In [10]:
batch_spec

{<Batch.inputs: 0>: [1, None, None, 3],
 <Batch.part_score_targets: 1>: [1, None, None, 1],
 <Batch.part_score_weights: 2>: [1, None, None, 1],
 <Batch.locref_targets: 3>: [1, None, None, 2],
 <Batch.locref_mask: 4>: [1, None, None, 2]}

## What is batch, enqueue_op and placeholders (output of setup_preloading)

In [11]:
from deeplabcut.pose_estimation_tensorflow.train import setup_preloading
# def setup_preloading(batch_spec):
#     placeholders = {name: tf.placeholder(tf.float32, shape=spec) for (name, spec) in batch_spec.items()}
#     names = placeholders.keys()
#     placeholders_list = list(placeholders.values())

#     QUEUE_SIZE = 20

#     q = tf.FIFOQueue(QUEUE_SIZE, [tf.float32]*len(batch_spec))
#     enqueue_op = q.enqueue(placeholders_list)
#     batch_list = q.dequeue()

#     batch = {}
#     for idx, name in enumerate(names):
#         batch[name] = batch_list[idx]
#         batch[name].set_shape(batch_spec[name])
#     return batch, enqueue_op, placeholders
batch, enqueue_op, placeholders = setup_preloading(batch_spec)

**batch** is a dictionary analogous to batch_spec that contains tf.tensors of class `tf.Tensor 'fifo_queue_1_Dequeue:i'`. Length = to the lists in batch_spec. float32.
dequeue tensors consist of the data collected after a dequeueing process in a TF queue. TF queues are the way TF provides support for multithreading (parallel) computing. It's an abstraction representing data tensors that support multithreading, but they can be treated as data or data batches.

In [12]:
batch

{<Batch.inputs: 0>: <tf.Tensor 'fifo_queue_Dequeue:0' shape=(1, ?, ?, 3) dtype=float32>,
 <Batch.part_score_targets: 1>: <tf.Tensor 'fifo_queue_Dequeue:1' shape=(1, ?, ?, 1) dtype=float32>,
 <Batch.part_score_weights: 2>: <tf.Tensor 'fifo_queue_Dequeue:2' shape=(1, ?, ?, 1) dtype=float32>,
 <Batch.locref_targets: 3>: <tf.Tensor 'fifo_queue_Dequeue:3' shape=(1, ?, ?, 2) dtype=float32>,
 <Batch.locref_mask: 4>: <tf.Tensor 'fifo_queue_Dequeue:4' shape=(1, ?, ?, 2) dtype=float32>}

**enqueue_op** is a tf.Operation object:
`<tf.Operation 'fifo_queue_1_enqueue' type=QueueEnqueueV2>`

**placeholders** is a dictionary like **batch** storing tensors of class a `tf.Tensor 'Placeholder_5:0'`. Length the same and float 32

In [13]:
placeholders

{<Batch.inputs: 0>: <tf.Tensor 'Placeholder:0' shape=(1, ?, ?, 3) dtype=float32>,
 <Batch.part_score_targets: 1>: <tf.Tensor 'Placeholder_1:0' shape=(1, ?, ?, 1) dtype=float32>,
 <Batch.part_score_weights: 2>: <tf.Tensor 'Placeholder_2:0' shape=(1, ?, ?, 1) dtype=float32>,
 <Batch.locref_targets: 3>: <tf.Tensor 'Placeholder_3:0' shape=(1, ?, ?, 2) dtype=float32>,
 <Batch.locref_mask: 4>: <tf.Tensor 'Placeholder_4:0' shape=(1, ?, ?, 2) dtype=float32>}

To sum up: batch, batch_spec and placeholders are dictionaries with 5 entries. Every key is a Batch object (Enum class). The 5 keys are shared by all 3 dictionaries.
* batch stores **TF queue_Dequeue tensors**
* batch_spec stores **Python lists**
* placeholders stores **TF Placeholder tensors**

## What are the 5 keys??

## What is losses?

Losses is a dict with three different losses, each a tensor
1. **part_loss**: `<tf.Tensor 'sigmoid_cross_entropy_loss/value:0' shape=() dtype=float32>`
2. **locref_loss**: `<tf.Tensor 'mul:0' shape=() dtype=float32>`
3. **total_loss**: `<tf.Tensor 'add:0' shape=() dtype=float32>`

It's the output of the `train` method in the PoseNet class.
  * `PoseNet.train(batch)
  
Their value is tracked with the `summary.scalars`

In [14]:
from deeplabcut.pose_estimation_tensorflow.dataset.pose_dataset import Batch
print(batch[Batch.part_score_weights])
print(batch.keys())

Tensor("fifo_queue_Dequeue:2", shape=(1, ?, ?, 1), dtype=float32)
dict_keys([<Batch.inputs: 0>, <Batch.part_score_targets: 1>, <Batch.part_score_weights: 2>, <Batch.locref_targets: 3>, <Batch.locref_mask: 4>])


In [15]:
from deeplabcut.pose_estimation_tensorflow.nnet.net_factory import pose_net
myobject = pose_net(cfg)
losses = myobject.train(batch)

## Looking deeper into the pose_net(cfg).train(batch) statement
This statement does the following:

* Instantiate an object of class PoseNet taking the cfg file as input. cfg is assigned to self
* Then, the train method in the object is called with batch as input

**The train() method** runs the get_net() method with the value under Batch.input key in the batch dictionary (more details in comments).
This gives us heads, which is a dictionary with 2 keys, `part_pred` and `locref` (1). The values are `slim.conv2d_transpose` objects, the output of `prediction_layer` i.e. they represent prediction layers, one for each different loss.
The resut of the function computes the loss based on the said prediction layers contained in heads.




1. `part_pred_interm` is a third key available only if `location_refinement` is true

In [16]:
print(cfg.location_refinement)
print(cfg.intermediate_supervision)
print(cfg.locref_huber_loss)


True
False
True


In [17]:
# def get_net(self, inputs):
#     # get_net calls extract_features and prediction layers
#     net, end_points = self.extract_features(inputs)
#     return self.prediction_layers(net, end_points)

def train(self, batch):
    cfg = self.cfg

    # call to the get_net() function defined above
    # * loads the resnet50 function (based on the cfg params)
    # * performs a substraction on the mean of the images
    # * adds prediction(s) layer.
    #    * The size of the layer depends on the number of features tracked.
    #    *  The number of prediction layers depends on the type of losses used (up to three different ones)
    
    # This is achieved with 2 functions: extract_features() and prediction_layers()
    # * extract_features() is just a customised call to resnet_v1 that
    #   - makes easy selection of the right resnet (50, 100, ...)
    #   - substracts the mean as stated above
    #   - modifies the default arguments of the resnet generator:
    #     global_pool=False, output_stride=16,is_training=False.
    # See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/slim/python/slim/nets/resnet_v1.py
    
    # * prediction_layers()
    #   - calls prediction_layer() for every loss being computed
    #   - fine tunes the call to prediction layer depending on the number of outputs 
    #     in part_pred loss, n_outs = n_features tracked.
    #     in locref, n_outs = n_features tracked * 2   
    heads = self.get_net(batch[Batch.inputs])
    # the output is a dictionary with every key being a prediction layer i.e. the very tip of the TF graph

    weigh_part_predictions = cfg.weigh_part_predictions
    part_score_weights = batch[Batch.part_score_weights] if weigh_part_predictions else 1.0

    def add_part_loss(pred_layer):
        return tf.losses.sigmoid_cross_entropy(batch[Batch.part_score_targets], # ground truth (labels/targets)
                                                   heads[pred_layer],           # prediction
                                                   part_score_weights)

    loss = {}
    loss['part_loss'] = add_part_loss('part_pred')
    total_loss = loss['part_loss']
    # intermediate_supervision (I don't know yet) is False in the reaching mackenzie example
    if cfg.intermediate_supervision:
        loss['part_loss_interm'] = add_part_loss('part_pred_interm')
        total_loss = total_loss + loss['part_loss_interm']

        
    # location_refinement (i.e. location of the features) is True in the reaching mackenzie example
    if cfg.location_refinement:
        locref_pred = heads['locref']
        locref_targets = batch[Batch.locref_targets]
        locref_weights = batch[Batch.locref_mask]

        loss_func = losses.huber_loss if cfg.locref_huber_loss else tf.losses.mean_squared_error
        loss['locref_loss'] = cfg.locref_loss_weight * loss_func(locref_targets, locref_pred, locref_weights)
        total_loss = total_loss + loss['locref_loss']

        # loss['total_loss'] = slim.losses.get_total_loss(add_regularization_losses=params.regularize)
    loss['total_loss'] = total_loss
    return loss

```
def extract_features(self, inputs):
        net_fun = net_funcs[self.cfg.net_type]

        mean = tf.constant(self.cfg.mean_pixel,
                           dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
        im_centered = inputs - mean

        # The next part of the code depends upon which tensorflow version you have.
        vers = tf.__version__
        vers = vers.split(".") #Updated based on https://github.com/AlexEMG/DeepLabCut/issues/44
        if int(vers[0])==1 and int(vers[1])<4: #check if lower than version 1.4.
            with slim.arg_scope(resnet_v1.resnet_arg_scope(False)):
                net, end_points = net_fun(im_centered,
                                          global_pool=False, output_stride=16)
        else:
            with slim.arg_scope(resnet_v1.resnet_arg_scope()):
                net, end_points = net_fun(im_centered,
                                          global_pool=False, output_stride=16,is_training=False)

        return net,end_points

def prediction_layers(self, features, end_points, reuse=None):
    cfg = self.cfg

    num_layers = re.findall("resnet_([0-9]*)", cfg.net_type)[0]
    layer_name = 'resnet_v1_{}'.format(num_layers) + '/block{}/unit_{}/bottleneck_v1'

    out = {}
    with tf.variable_scope('pose', reuse=reuse):
        out['part_pred'] = prediction_layer(cfg, features, 'part_pred',
                                            cfg.num_joints)
        if cfg.location_refinement:
            out['locref'] = prediction_layer(cfg, features, 'locref_pred',
                                             cfg.num_joints * 2)
        if cfg.intermediate_supervision:
            interm_name = layer_name.format(3, cfg.intermediate_supervision_layer)
            block_interm_out = end_points[interm_name]
            out['part_pred_interm'] = prediction_layer(cfg, block_interm_out,
                                                       'intermediate_supervision',
                                                       cfg.num_joints)

    return out
```    

In [18]:
# losses = pose_net(cfg).train(batch)

## After the loss is computed...

Add the losses to the TF tracking system

In [19]:
for k, t in losses.items():
    tf.summary.scalar(k, t)
merged_summaries = tf.summary.merge_all()
total_loss = losses["total_loss"]

In [20]:
## Variables to restore?

In [21]:
import tensorflow.contrib.slim as slim
variables_to_restore = slim.get_variables_to_restore(include=["resnet_v1"])
restorer = tf.train.Saver(variables_to_restore)
saver = tf.train.Saver(max_to_keep=max_to_keep)    

Huge amount of variables extracted from the pretrained ResNet50 print(len(variables_to_restore))
TODO: Google restorers and savers!

In [22]:
print(max_to_keep)
variables_to_restore

5


[<tf.Variable 'resnet_v1_50/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/conv1/BatchNorm/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/conv1/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/conv1/BatchNorm/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/conv1/BatchNorm/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/gamma:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/moving_mean:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'resnet_v1_50/block1/unit_1/bottleneck_v1/shortcut/BatchNorm/moving_vari

## Create a session and preload it
i.e. give it instructions on how to load data

What it does:

1. Instantiates a Coordinator object, which ensures that all parallel threads are ended simultaneously and merges them to have an integrated output: https://adventuresinmachinelearning.com/introduction-tensorflow-queuing/

2. Declares the threading process, which is defined by load_and_enqueue():

```
def load_and_enqueue(sess, enqueue_op, coord, dataset, placeholders):
    while not coord.should_stop():
        batch_np = dataset.next_batch()
        food = {pl: batch_np[name] for (name, pl) in placeholders.items()}
        sess.run(enqueue_op, feed_dict=food)
```

`load_and_enqueue()` runs the enqueue operation (enqueue_op) using food. Food is a dictionary where every entry is a a placeholder (pl) from the placeholders dict. The placeholders are loaded with the data from one batch. The function thus
1. Retrieves the next batch (batch_np) from the dataset object, as long as the coordinator does not signal the end of the computations.
2. Fills the placeholders with the data of that batch and puts them in a new dict food
3. Enqueue the data of the batch in the TF queueing sytem

This mechanism is wrapped by the `Thread()` class, which makes every thread run the operations defined in `load_and_enqueue()` in parallel.
The input arguments of `start_preloading()` are passed untouched to every thread to do with them as explained above.

```
def start_preloading(sess, enqueue_op, dataset, placeholders):
    coord = tf.train.Coordinator()

    t = threading.Thread(target=load_and_enqueue,
                         args=(sess, enqueue_op, coord, dataset, placeholders))
    t.start()

    return coord, t
```

In a nutshell, as long as the threads are not to be stopped, the next batch is loaded, and the placeholders defined above are loaded with data. Finally, they are passed to the enqueue operation to be run by the TF session.

3. Starts the threading process

This returns the coordinator and thread objects


In [23]:
sess = tf.Session()
writer = tf.summary.FileWriter('logs', sess.graph)

# from deeplabcut.pose_estimation_tensorflow.train import load_and_enqueue      
from deeplabcut.pose_estimation_tensorflow.train import start_preloading
coord, thread = start_preloading(sess, enqueue_op, dataset, placeholders)

## Declare a loss optimization method (specified in the cfg file). Options are SGD or Adam

The total loss will be optimized. The total loss is the sum of all the computed losses

In [ ]:
from deeplabcut.pose_estimation_tensorflow.train import get_optimizer

In [ ]:
train_writer = tf.summary.FileWriter(cfg.log_dir, sess.graph)

In [ ]:
learning_rate, train_op = get_optimizer(total_loss, cfg)

In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [24]:
# Restore variables from disk.
restorer.restore(sess, cfg.init_weights)
if maxiters==None:
    max_iter = int(cfg.multi_step[-1][1])
else:
    max_iter = min(int(cfg.multi_step[-1][1]),int(maxiters))
    #display_iters = max(1,int(displayiters))
    print("Max_iters overwritten as",max_iter)

if displayiters==None:
    display_iters = max(1,int(cfg.display_iters))
else:
    display_iters = max(1,int(displayiters))
    print("Display_iters overwritten as",display_iters)

if saveiters==None:
     save_iters=max(1,int(cfg.save_iters))

else:
    save_iters=max(1,int(saveiters))
    print("Save_iters overwritten as",save_iters)

INFO:tensorflow:Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Max_iters overwritten as 100
Display_iters overwritten as 100
Save_iters overwritten as 100


In [25]:
from deeplabcut.pose_estimation_tensorflow.train import LearningRate

# class LearningRate(object):
#     def __init__(self, cfg):
#         self.steps = cfg.multi_step
#         self.current_step = 0

#     def get_lr(self, iteration):
#         lr = self.steps[self.current_step][0]
#         if iteration == self.steps[self.current_step][1]:
#             self.current_step += 1

#         return lr

    
cum_loss = 0.0
lr_gen = LearningRate(cfg)
stats_path = Path(config_yaml).with_name('learning_stats.csv')
lrf = open(str(stats_path), 'w')

In [26]:
lr_gen

In [29]:
print("Training parameter:")
print(cfg)
print("Starting training....")
for it in range(max_iter+1):
    print(it)
    current_lr = lr_gen.get_lr(it)
    [_, loss_val, summary] = sess.run([train_op, total_loss, merged_summaries],
                                      feed_dict={learning_rate: current_lr})
    cum_loss += loss_val
    train_writer.add_summary(summary, it)
  
    if it % display_iters == 0 and it>0:
        average_loss = cum_loss / display_iters
        cum_loss = 0.0
        logging.info("iteration: {} loss: {} lr: {}"
                     .format(it, "{0:.4f}".format(average_loss), current_lr))
        lrf.write("{}, {:.5f}, {}\n".format(it, average_loss, current_lr))
        lrf.flush()

    # Save snapshot
    if (it % save_iters == 0 and it != 0) or it == max_iter:
        model_name = cfg.snapshot_prefix
        saver.save(sess, model_name, global_step=it)

Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/antortjim/MEGA/FlySleepLab/FlyRecognition/DLC/examples/Reaching-Mackenzie-2018-08-30/dlc-models/iteration-0/ReachingAug30-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'all_joints': [[0], [1], [2], [3]], 

RuntimeError: Attempted to use a closed Session.

In [28]:
lrf.close()
sess.close()
writer.close()

coord.request_stop()
coord.join([thread])
#return to original path.
os.chdir(str(start_path))

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1278, in _do_call
    return fn(*args)
  File "/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1263, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/antortjim/anaconda3/envs/DLC/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1350, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[Node: fifo_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](fifo_queue, _arg_Placeholder_0_0, _arg_Placeholder_1_0_1, _arg_Placeholder_2_0_2, _arg_Placeholder_3_0_3, _arg_Placeholder_4_0_4)]]

During handling of the ab

In [33]:
start_path

'/home/antortjim/MEGA/FlySleepLab/DeepLabCut/examples'

## Test the autotune setting in TF 1.10.0
Pending